# Exploratory Data Analysis (EDA)

Step 1: Load the dataset and develop an initial understanding
    - Inspect shape, preview rows, review variable types, isolate the target.

Step 2: Assess missing values and remove variables with excessive missingness
    - Compute missing-value percentages, drop columns over threshold.

Step 3: Correct variable types and recode categories
    - Convert numeric→categorical where needed, encode ordinal variables, fix dtype inconsistencies.

Step 4: Univariate analysis

Step 5: Bivariate analysis

Step 6: Multivariate checks and feature interactions

Step 7: Prepare final feature lists for modeling

In [2]:
# Define necessary functions
def encode_ordinal(df, columns, order):
    for column in columns:
        df[column] = (
            pd.Categorical(df[column], categories=order, ordered=True)
            .codes
            .astype('float')
        )
        df[column] = df[column].replace(-1, np.nan)
    return df

In [3]:
# Load libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.api.types as pd_types

# Print file names in directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [4]:
# specify input paths
train_file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
submission_file_path = '/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv'

# load as pandas data frames
df = pd.read_csv(train_file_path)
sub_df = pd.read_csv(submission_file_path)

# separate variables
target = df["SalePrice"]
df = df.drop(columns=["SalePrice"])

# display options
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

df.head(20)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,...,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,...,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,...,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,...,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,...,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,...,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,...,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,...,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,...,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,...,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,...,Typ,2,TA,Attchd,1939.0,RFn,1,205,Gd,TA,Y,0,4,0,0,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

A quick view of the data suggests the following:

(1) Some variables, like 'GarageQual', may be on an ordinal scale but use categories (e.g., TA, Fa, Gd) while other variable denote a class, such as 'MSSubClass', but use integers. This suggests that some variables may need to be recoded

(2) In addition, other variables, like '', show missing data in these first entries. Therefore, variables may need to be dropped from the dataset. I use a threshold of 30% to decide which to exclude.

In [6]:
# Drop columns with missing values

# calculate percentage of missing values per column
pct_missing = (df.isna().sum() / len(df) * 100).round(2)
drop_vars = pct_missing[pct_missing > 30].index.tolist() # get columns names above threshold

# Show columns with highest amount of missing values
print(drop_vars)

# Drop columns with too many missing values from the dataset
df = df.drop(columns=drop_vars)
pct_missing = (df.isna().sum() / len(df) * 100).round(2)
print(pct_missing.sort_values(ascending=False)) # confirm that columns were dropped

['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
LotFrontage      17.74
GarageCond        5.55
GarageQual        5.55
GarageType        5.55
GarageYrBlt       5.55
                 ...  
MiscVal           0.00
MoSold            0.00
YrSold            0.00
SaleType          0.00
SaleCondition     0.00
Length: 74, dtype: float64


Missing values for the remaining columns will later be imputed using the most common or median value via scikit-learn pipelines.

In [7]:
# Recode dtype based on variable descriptions #

# Change MSSubClass from numeric to unordered object: 
df['MSSubClass'] = df['MSSubClass'].astype('object')  # or 'object'

# Recode quality ratings from unordered objects to ordered categories:
qual_order = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
columns = ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual']
df = encode_ordinal(df, columns, qual_order)

qual_order = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
columns = ['BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond']
df = encode_ordinal(df, columns, qual_order)

qual_order = ['NA', 'No', 'Mn', 'Av', 'Gd']
columns = ['BsmtExposure']
df = encode_ordinal(df, columns, qual_order)

qual_order = ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
columns = ['BsmtFinType1', 'BsmtFinType2']
df = encode_ordinal(df, columns, qual_order)

# Convert Year from a float to an integer
df['GarageYrBlt'] = df['GarageYrBlt'].astype('float')

df.dtypes.value_counts()

int64      33
object     27
float64    14
Name: count, dtype: int64

These changes target some obvious errors, such as 'MSSubClass' being an integer despite representing unordered categories, as well as categorical variables with unordered variables that should be ordinal.
After recoding the variables, there are 33 (int64) + 14 (float64) numeric and 27 (object) categorical features